In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader

from sklearn.model_selection import KFold
from sklearn.metrics import *

In [2]:
dataset_path = 'output_folder'

In [3]:
transform = transforms.Compose(
    [transforms.Resize(size = (220,220)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5], std=[0.5])
    ]
)

In [4]:
dataset = datasets.ImageFolder(dataset_path, transform = transform)

In [5]:
print("Informações sobre o Dataset: \n\n", dataset)
print("\nRótulos: ", dataset.class_to_idx)

Informações sobre o Dataset: 

 Dataset ImageFolder
    Number of datapoints: 24
    Root location: output_folder
    StandardTransform
Transform: Compose(
               Resize(size=(220, 220), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )

Rótulos:  {'ANOMALIA': 0, 'NORMAL': 1}


In [6]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    
    nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),

    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    
    nn.Flatten(),
    
    nn.Linear(64 * 110 * 110, 100),
    nn.Dropout(p=0.4),
    nn.ReLU(),
    
    nn.Linear(100, 2)
)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=774400, out_features=100, bias=True)
  (12): Dropout(p=0.4, inplace=False)
  (13): ReLU()
  (14): Linear(in_features=100, out_features=2, bias=True)
)

In [8]:
num_epoch = 35
learning_rate = 0.001
k_folds = 5

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
kf = KFold(n_splits=k_folds, shuffle=True)

In [9]:
results = {}
train_array = []
test_array = []

for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f'Fold {fold+1}/{k_folds}')

    train_array.append(train_idx)
    test_array.append(test_idx)

    train_subset = Subset(dataset, train_idx)
    test_subset = Subset(dataset, test_idx)

    trainloader = DataLoader(train_subset, shuffle=True)
    testloader = DataLoader(test_subset, shuffle=False)

    for epoch in range(num_epoch):
        model.train()
        running_train_loss = 0.0

        for inputs, labels in trainloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()

        train_loss = running_train_loss / len(trainloader)
        _, predicted = torch.max(outputs.data, 1)
        
        acc_treino = accuracy_score(labels.cpu().detach().numpy(), predicted.cpu().detach().numpy())

        print(f"Época {epoch + 1}/{num_epoch} - Perda no treinamento: {train_loss:.6f} - Acurácia: {acc_treino:.4f}")

    model.eval()

    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
        
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
    
    acc_teste = accuracy_score(labels.cpu().detach().numpy(), predicted.cpu().detach().numpy())
    print(f'Acc para o Fold {fold+1}: {acc_teste:.4f}')
    results[fold] = acc_teste

    print("!!!Teste finalizado!!!")

print(f'\nResultados K-Fold: {results}')
print(f'Média Acc: {sum(results.values()) / k_folds}')

Fold 1/5
Época 1/35 - Perda no treinamento: 78.170804 - Acurácia: 0.0000
Época 2/35 - Perda no treinamento: 0.000000 - Acurácia: 1.0000
Época 3/35 - Perda no treinamento: 1.893992 - Acurácia: 1.0000
Época 4/35 - Perda no treinamento: 12.928587 - Acurácia: 1.0000
Época 5/35 - Perda no treinamento: 0.000000 - Acurácia: 1.0000
Época 6/35 - Perda no treinamento: 25.690038 - Acurácia: 1.0000
Época 7/35 - Perda no treinamento: 0.000000 - Acurácia: 1.0000
Época 8/35 - Perda no treinamento: 0.000000 - Acurácia: 1.0000
Época 9/35 - Perda no treinamento: 0.000000 - Acurácia: 1.0000
Época 10/35 - Perda no treinamento: 0.000000 - Acurácia: 1.0000
Época 11/35 - Perda no treinamento: 0.000000 - Acurácia: 1.0000
Época 12/35 - Perda no treinamento: 45.339728 - Acurácia: 1.0000
Época 13/35 - Perda no treinamento: 17.063461 - Acurácia: 1.0000
Época 14/35 - Perda no treinamento: 0.000000 - Acurácia: 1.0000
Época 15/35 - Perda no treinamento: 17.141561 - Acurácia: 1.0000
Época 16/35 - Perda no treinamento

In [10]:
train_array

[array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 18, 19, 20,
        21, 22]),
 array([ 0,  1,  2,  4,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 23]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7, 10, 11, 12, 14, 15, 17, 18, 20, 21,
        22, 23]),
 array([ 0,  1,  2,  3,  5,  6,  8,  9, 10, 11, 13, 15, 16, 17, 18, 19, 21,
        22, 23]),
 array([ 0,  1,  3,  4,  5,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 19, 20,
        21, 22, 23])]

In [11]:
test_array

[array([ 0,  1, 15, 17, 23]),
 array([ 3,  5, 11, 21, 22]),
 array([ 8,  9, 13, 16, 19]),
 array([ 4,  7, 12, 14, 20]),
 array([ 2,  6, 10, 18])]